In [1]:
from datetime import date
import time
import sys
import warnings
warnings.filterwarnings('ignore') 
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from astropy.table import Table
from astroquery.sdss import SDSS
from matplotlib import pyplot as plt

import astropy.units as u
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from astropy.coordinates import SkyCoord

# Spectroscopic Data

In [45]:
query_sdss = """SELECT TOP 270000
        s.specobjid, s.ra, s.dec, g.h_alpha_flux, g.h_beta_flux, 
        g.sii_6717_flux, g.sii_6731_flux, g.nii_6548_flux, g.nii_6584_flux,
        g.oi_6300_flux, g.oii_3726_flux, g.oii_3729_flux, g.oiii_5007_flux,
        g.h_alpha_eqw, p.u,p.g,p.r,p.i,p.z,s.z as redshift,ID.bptclass
                
        FROM SpecObjAll AS s
        JOIN galSpecExtra AS ID ON ID.specObjID = s.specObjID
        JOIN galSpecLine AS g ON g.specObjID = s.specObjID
        JOIN PhotoObj AS p ON p.objID = s.bestObjID
                
        WHERE 
        s.z <=1
        AND s.class IN ('QSO','GALAXY')
        AND g.h_alpha_flux >0 AND g.h_beta_flux >0
        AND g.sii_6717_flux >0 AND g.sii_6731_flux >0 
        AND g.nii_6548_flux >0 AND g.nii_6584_flux >0
        AND g.oii_3726_flux >0 AND g.oii_3729_flux >0
        AND g.oi_6300_flux >0 AND g.oiii_5007_flux >0         
        AND p.u >0 AND p.g >0 AND p.r >0 AND p.i >0 AND p.z >0   
        --ORDER BY redshift DESC
        """

data_sdss = SDSS.query_sql(query_sdss)
t=Table(data_sdss)
sdss_df=t.to_pandas()
sdss_df['ra'] = sdss_df['ra'].astype(str) 
sdss_df['dec'] = sdss_df['dec'].astype(str) 
sdss_df['Full Name'] = sdss_df[['ra','dec']].agg(' - '.join, axis=1) 
sdss_df = sdss_df.drop_duplicates(subset = ['Full Name']).reset_index(drop=True)
sdss_df

,specobjid,ra,dec,h_alpha_flux,h_beta_flux,sii_6717_flux,sii_6731_flux,nii_6548_flux,nii_6584_flux,oi_6300_flux,oii_3726_flux,oii_3729_flux,oiii_5007_flux,h_alpha_eqw,u,g,r,i,z,redshift
0,299490502078654464,146.62857,-0.76513683,99.40,25.64,35.33,21.66,17.10,51.59,8.49,39.06,27.10,21.44,-5.13,19.50,17.59,16.68,16.21,15.85,0.06
1,299491051834468352,146.63167,-0.98827781,127.31,38.33,30.27,25.72,8.55,25.79,6.27,48.75,47.95,35.47,-21.19,19.48,18.33,17.84,17.53,17.42,0.05
2,299491326712375296,146.91945,-0.99049175,13.32,7.02,6.76,4.06,4.48,13.53,4.22,5.79,7.51,8.56,-1.84,20.44,18.65,17.47,17.00,16.67,0.21
3,299492700632147968,146.59272,-0.76025604,211.50,53.18,41.64,28.48,25.31,76.35,7.21,32.12,27.30,15.60,-14.73,19.07,17.62,16.97,16.58,16.35,0.07
4,299492975979816960,146.9635,-0.75935173,21.56,3.30,12.78,7.07,5.25,15.84,3.17,2.51,14.52,17.70,-1.43,20.32,18.51,17.62,17.21,16.91,0.10
5,299493800613537792,146.85983,-0.80890165,11.44,4.32,10.43,6.20,6.42,19.36,0.59,17.56,0.39,11.26,-1.19,20.78,18.65,17.31,16.71,16.22,0.13
6,299494350369351680,146.8577,-0.66285236,29.85,10.79,0.62,5.91,3.35,10.10,3.17,7.61,15.98,0.24,-8.90,19.90,18.33,17.61,17.21,17.15,0.08
7,299495724758886400,147.02336,-0.16009352,37.99,3.40,5.21,0.86,4.75,14.31,1.64,2.70,5.25,4.28,-5.10,20.49,18.88,17.79,17.29,16.88,0.22
8,299496274514700288,146.95607,-0.34230044,77.05,22.93,36.78,37.11,19.52,58.87,12.49,40.41,30.49,20.21,-4.77,21.02,18.92,17.61,17.03,16.59,0.13
9,299496823800752128,146.9201,-0.30646208,58.96,8.35,4.66,10.21,9.99,30.14,0.94,6.31,16.76,3.66,-5.15,20.13,18.56,17.49,16.99,16.62,0.13


In [46]:
ejex1=np.log10((sdss_df['nii_6548_flux']+sdss_df['nii_6584_flux'])/sdss_df['h_alpha_flux'])
ejex2=np.log10((sdss_df['sii_6717_flux']+sdss_df['sii_6731_flux'])/sdss_df['h_alpha_flux'])
ejex3=np.log10(sdss_df['oi_6300_flux']/sdss_df['h_alpha_flux'])
ejex4=np.log10((sdss_df['oii_3726_flux']+sdss_df['oii_3729_flux'])/sdss_df['h_alpha_flux'])
ejex5=np.log10((sdss_df['oii_3726_flux']+sdss_df['oii_3729_flux'])/sdss_df['h_beta_flux'])

ejey2=np.log10(np.abs(sdss_df['h_alpha_eqw']))
ejey=np.log10((sdss_df['oiii_5007_flux'])/(sdss_df['h_beta_flux']))

In [47]:
sdss_df['OIII/Hβ']=ejey
sdss_df['NII/Hα']=ejex1
sdss_df['SII/Hα']=ejex2
sdss_df['OI/Hα']=ejex3
sdss_df['OII/Hα']=ejex4
sdss_df['OII/Hβ']=ejex5
sdss_df['W_Hα']=ejey2

#U-B
sdss_df['U']=(sdss_df['u']-0.0682)-(0.0140*(sdss_df['u']-sdss_df['g']-1.2638))
sdss_df['B']=(sdss_df['u']-1.0286)-(0.7981*(sdss_df['u']-sdss_df['g']-1.2638))
sdss_df['U-B']=(sdss_df['U']-sdss_df['B'])

#UGRIZ
sdss_df['u-r']=sdss_df['u']-sdss_df['r']
sdss_df['g-z']=sdss_df['g']-sdss_df['z']

data=sdss_df[['OIII/Hβ','NII/Hα','SII/Hα','OI/Hα','OII/Hα','OII/Hβ','U-B','u-r','g-z','W_Hα']].dropna()
data

,OIII/Hβ,NII/Hα,SII/Hα,OI/Hα,OII/Hα,OII/Hβ,U-B,u-r,g-z,W_Hα
0,-0.077693,-0.160493,-0.241588,-1.068479,-0.176791,0.411677,1.467085,2.82,1.74,0.710117
1,-0.033678,-0.569062,-0.356752,-1.307595,-0.119436,0.401888,0.871169,1.64,0.91,1.326131
2,0.086137,0.131009,-0.090277,-0.499192,-0.000653,0.277515,1.372993,2.97,1.98,0.264818
3,-0.532624,-0.318160,-0.479468,-1.467375,-0.551378,0.048184,1.106399,2.10,1.27,1.168203
4,0.729459,-0.009572,-0.035888,-0.832589,-0.102434,0.712701,1.388675,2.70,1.60,0.155336
5,0.416055,0.352857,0.162466,-1.287574,0.195638,0.618581,1.639587,3.47,2.43,0.075547
6,-1.652810,-0.346222,-0.660031,-0.973885,-0.102216,0.339706,1.200491,2.29,1.18,0.949390
7,0.099965,-0.299546,-0.796481,-1.364825,-0.679302,0.368888,1.231855,2.70,2.00,0.707570
8,-0.054838,0.007488,-0.018187,-0.790210,-0.036126,0.490242,1.616064,3.41,2.33,0.678518
9,-0.358205,-0.167088,-0.598247,-1.797430,-0.407510,0.441361,1.200491,2.64,1.94,0.711807


In [ ]:
dataset_test = data.values
len(dataset_test)

In [ ]:
train=pd.read_csv('Base_Training.csv')
train

In [ ]:
'''dataset_test = train
name=['OIII/Hβ','NII/Hα','SII/Hα','OI/Hα','OII/Hα','OII/Hβ','U-B','u-r','g-z','W_Hα','a']
dataset_test.columns = name
dataset_test=dataset_test.iloc[:,0:10]
dataset_test'''

In [ ]:
from tensorflow.keras.utils import to_categorical
dataset_train = train.values
x_train = dataset_train[:,0:10].astype(float)
y = dataset_train[:,10]
y_train=to_categorical(y)

In [ ]:
from keras.models import Sequential
from keras.layers import LeakyReLU

model = Sequential()
model.add(Dense(50, input_dim=10, activation=LeakyReLU(alpha=0.1), kernel_initializer='he_normal'))
model.add(Dense(20, activation=LeakyReLU(alpha=0.01)))
model.add(Dense(10, activation=LeakyReLU(alpha=0.1)))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
hist=model.fit(x_train,y_train, batch_size=50, epochs=100, validation_split=0.1)

In [ ]:
x_test = model.predict(dataset_test)
y_test_class = np.argmax(x_test, axis=1)
#np.set_printoptions(threshold=sys.maxsize)
x_test

In [ ]:
y_test_class

In [ ]:
df_predicciones = pd.DataFrame({'prediccion' : y_test_class})
df_predicciones

In [ ]:
confusion=confusion_matrix(y, y_test_class)

# Visualizar la matriz de confusión
plt.imshow(confusion, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matriz de Confusión')
plt.colorbar()

classes = ['Comp','LINER','Seyfert','Starburst']
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

thresh = confusion.max() / 2.
for i in range(confusion.shape[0]):
    for j in range(confusion.shape[1]):
        plt.text(j, i, format(confusion[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if confusion[i, j] > thresh else "black")

plt.xlabel('Valores Predicción')
plt.ylabel('Valores Reales')
plt.tight_layout()
plt.show()

In [ ]:
scores = model.evaluate(x_train, y_train, verbose=0)
print('Precisión del modelo: {}\n Error del modelo: {}'.format(scores[1], 1 - scores[1]))   

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Modelo de Precisión')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Testeo'], loc='lower right')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Modelo de Pérdida')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Testeo'], loc='upper right')
plt.show()

In [ ]:
weights = model.layers[0].get_weights()[0]

# Grafica un histograma de los pesos
plt.hist(weights.flatten(), bins=50, density=True)
plt.title("Distribución de pesos primera capa")
plt.xlabel("Valor del peso")
plt.ylabel("Frecuencia")
plt.show()
desviacion_estandar = np.std(weights)
media = np.mean(weights)
print("Desviación Estándar de los Pesos:", desviacion_estandar)
print("Media de los Pesos:", media)

In [ ]:
weights = model.layers[1].get_weights()[0]

# Grafica un histograma de los pesos
plt.hist(weights.flatten(), bins=50, density=False)
plt.title("Distribución de pesos última capa")
plt.xlabel("Valor del peso")
plt.ylabel("Frecuencia")
plt.show()
desviacion_estandar = np.std(weights)
media = np.mean(weights)
print("Desviación Estándar de los Pesos:", desviacion_estandar)
print("Media de los Pesos:", media)

In [ ]:
column_names = ['OIII/Hβ','NII/Hα','SII/Hα','OI/Hα','OII/Hα','OII/Hβ','U-B','u-r','g-z','W_Hα']
df=pd.DataFrame(dataset_test,columns=column_names)
df['Predicted Class'] = pd.Series(y_test_class)
df

In [ ]:
comp=df['Predicted Class']==0
lin=df['Predicted Class']==1
sf=df['Predicted Class']==2
sb=df['Predicted Class']==3

In [ ]:
# Kewley
x1 = np.linspace(-3.6,0.469)
y1 = 1.19 + (0.61/(x1-0.47))

#Kauffman
x2 = np.linspace(-3.6,0.0) 
y2 = 1.3 + (0.61/(x2-0.05))

#Cid-Fernandes
x3 = np.linspace(-0.2,4.0) 
y3 = 0.48 +(1.01*x3)

fig = plt.figure(figsize=(15,10))
ax0 = fig.add_subplot(1,1,1)
ax0.plot(x1, y1, '--' , color='dimgray',lw=1.5, label='Kewley') 
ax0.plot(x2, y2, '-' , color='dimgray', lw=1.5, label='Kauffman') 
ax0.plot(x3, y3, '-.' , color='dimgray', lw=1.5, label='Cid. Fernandes') 

ax0.scatter(df['NII/Hα'], df['OIII/Hβ'],comp*50, marker='*',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax0.scatter(df['NII/Hα'], df['OIII/Hβ'],lin*40, marker='o',facecolors="none",edgecolors="orange",label='LINER') 
ax0.scatter(df['NII/Hα'], df['OIII/Hβ'],sf*40, marker='^',facecolors="none",edgecolors="dodgerblue",label='Seyfert') 
ax0.scatter(df['NII/Hα'], df['OIII/Hβ'],sb*40, marker='d',facecolors="none",edgecolors="red",label='Starburst') 
ax0.scatter(train['NII/Hα(T)'], train['OIII/Hβ(T)'], c="black", s=0.05,label='Training Data') 

ax0.set_ylim(-2.5,2.5) 
ax0.set_xlim(-2,1) 
ax0.set_xlabel(r'Log(([NII]6548+[NII]6584)/Hα)',fontsize=16) 
ax0.set_ylabel(r'Log([OIII]5007/Hβ)',fontsize=16)
ax0.tick_params(axis='both', which='major', labelsize=12) 
plt.legend()
plt.show()

# EQW

In [ ]:
eqwx=df['NII/Hα']
eqwy=df['W_Hα']

fig, ax = plt.subplots(figsize=(8,6)) 

ax.scatter(eqwx,eqwy, lin*1, marker='.',facecolors="none",edgecolors="orange",label='LINER')
ax.scatter(eqwx,eqwy, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax.scatter(eqwx,eqwy, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax.scatter(eqwx,eqwy, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax.scatter(train['NII/Hα(T)'], train['W_Hα(T)'], c="black", s=0.02,label='Training Data') 
ax.axvline(x=-0.4, color='black', linestyle='-.', lw=0.5)
ax.hlines(y=0.778, xmin=-0.4, xmax=3, color='black', linestyle='-.', lw=0.5)
ax.hlines(y=-0.3, xmin=-4, xmax=7, color='black', linestyle='-.', lw=0.5)
x = np.linspace(-3,3)
y=(-1*x)-0.25
ax.plot(x,y,':', color='black',lw=1) 
ax.legend()
ax.set_xlim(-1.25,0.75)
ax.set_ylim(-1,2.5)
ax.set_xlabel(r'Log(NII/Hα)',fontsize=16) 
ax.set_ylabel(r'Log($W_{Hα}$)',fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.show()

fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].scatter(eqwx,eqwy, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax[1].scatter(eqwx,eqwy, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax[2].scatter(eqwx,eqwy, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax[3].scatter(eqwx,eqwy, lin*1, marker='.',facecolors="none",edgecolors="orange",label='Liner')
ax[0].axvline(x=-0.4, color='black', linestyle='-.', lw=0.5)
ax[0].hlines(y=0.778, xmin=-0.4, xmax=3, color='black', linestyle='-.', lw=0.5)
ax[0].hlines(y=-0.3, xmin=-4, xmax=7, color='black', linestyle='-.', lw=0.5)
ax[0].plot(x,y,':', color='black',lw=1) 
ax[1].axvline(x=-0.4, color='black', linestyle='-.', lw=0.5)
ax[1].hlines(y=0.778, xmin=-0.4, xmax=3, color='black', linestyle='-.', lw=0.5)
ax[1].hlines(y=-0.3, xmin=-4, xmax=7, color='black', linestyle='-.', lw=0.5)
ax[1].plot(x,y,':', color='black',lw=1) 
ax[2].axvline(x=-0.4, color='black', linestyle='-.', lw=0.5)
ax[2].hlines(y=0.778, xmin=-0.4, xmax=3, color='black', linestyle='-.', lw=0.5)
ax[2].hlines(y=-0.3, xmin=-4, xmax=7, color='black', linestyle='-.', lw=0.5)
ax[2].plot(x,y,':', color='black',lw=1) 
ax[3].axvline(x=-0.4, color='black', linestyle='-.', lw=0.5)
ax[3].hlines(y=0.778, xmin=-0.4, xmax=3, color='black', linestyle='-.', lw=0.5)
ax[3].hlines(y=-0.3, xmin=-4, xmax=7, color='black', linestyle='-.', lw=0.5)
ax[3].plot(x,y,':', color='black',lw=1) 
ax[0].set_xlim(-1,0.5)
ax[1].set_xlim(-1.5,0)
ax[2].set_xlim(-1,0.5)
ax[3].set_xlim(-0.5,0.75)
ax[0].set_ylim(-1,2.5)
ax[1].set_ylim(-1,2.5)
ax[2].set_ylim(-1,2.5)
ax[3].set_ylim(-1,2.5)
ax[0].set_xlabel(r'Log([NII]/Hα)')
ax[1].set_xlabel(r'Log([NII]/Hα)')
ax[2].set_xlabel(r'Log([NII]/Hα)')
ax[3].set_xlabel(r'Log([NII]/Hα)')
ax[0].set_ylabel(r'Log($W_{Hα}$)')
plt.show()

# Blue

In [ ]:
eqwx=df['OII/Hβ']
eqwy=df['OIII/Hβ']

x11 = np.linspace(-5,0.9)
y11 = 0.85 + (0.11/(x11-0.92))

x22 = np.linspace(0.7,4)
y22 = 0.95 * (x22-0.4)

x6 = np.linspace(-5,0.7)
y6 = 0.3+ (x6*0)

x7 = np.linspace(0.05, 1.07)
y7 = (-(x7 - 1.0)**2) - (0.1*x7) + 0.25
x8 = np.linspace(0.05 ,1.07)
y8 = ((x8 - 0.2)**2) - 0.6

fig, ax = plt.subplots(figsize=(8,6)) 
ax.scatter(eqwx,eqwy, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax.scatter(eqwx,eqwy, lin*1, marker='.',facecolors="none",edgecolors="orange",label='LINER')
ax.scatter(eqwx,eqwy, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax.scatter(eqwx,eqwy, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax.scatter(train['OII/Hβ(T)'], train['OIII/Hβ(T)'], c="black", s=0.02,label='Training Data') 
ax.plot(x11, y11, '--' , color='black',lw=1, label='Lamareille') 
ax.plot(x22, y22, '-' , color='black', lw=1, label='Lamareille') 
ax.hlines(y=0.3, xmin=-1.5, xmax=0.7, color='black', linestyle='-.', lw=0.5)
#ax.plot(x7, y7, '--' , color='black',lw=0.5)
#ax.plot(x8, y8, '--' , color='black',lw=0.5)
ax.legend()
ax.set_xlim(-1.5,2)
ax.set_ylim(-2,2)
ax.set_xlabel(r'Log([OII]/Hβ)',fontsize=16) 
ax.set_ylabel(r'Log([OIII]/Hβ)',fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.show()

fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].scatter(eqwx,eqwy, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax[1].scatter(eqwx,eqwy, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax[2].scatter(eqwx,eqwy, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax[3].scatter(eqwx,eqwy, lin*1, marker='.',facecolors="none",edgecolors="orange",label='Liner')
ax[0].plot(x11, y11, '--' , color='black',lw=1, label='Lamareille') 
ax[0].plot(x22, y22, '-' , color='black', lw=1, label='Lamareille') 
ax[0].hlines(y=0.3, xmin=-1.5, xmax=0.7, color='black', linestyle='-.', lw=0.5)
#ax[0].plot(x7, y7, '--' , color='black',lw=0.5)
#ax[0].plot(x8, y8, '--' , color='black',lw=0.5)
ax[1].plot(x11, y11, '--' , color='black',lw=1, label='Lamareille') 
ax[1].plot(x22, y22, '-' , color='black', lw=1, label='Lamareille') 
ax[1].hlines(y=0.3, xmin=-1.5, xmax=0.7, color='black', linestyle='-.', lw=0.5)
#ax[1].plot(x7, y7, '--' , color='black',lw=0.5)
#ax[1].plot(x8, y8, '--' , color='black',lw=0.5)
ax[2].plot(x11, y11, '--' , color='black',lw=1, label='Lamareille') 
ax[2].plot(x22, y22, '-' , color='black', lw=1, label='Lamareille') 
ax[2].hlines(y=0.3, xmin=-1.5, xmax=0.7, color='black', linestyle='-.', lw=0.5)
#ax[2].plot(x7, y7, '--' , color='black',lw=0.5)
#ax[2].plot(x8, y8, '--' , color='black',lw=0.5)
ax[3].plot(x11, y11, '--' , color='black',lw=1, label='Lamareille') 
ax[3].plot(x22, y22, '-' , color='black', lw=1, label='Lamareille') 
ax[3].hlines(y=0.3, xmin=-1.5, xmax=0.7, color='black', linestyle='-.', lw=0.5)
#ax[3].plot(x7, y7, '--' , color='black',lw=0.5)
#ax[3].plot(x8, y8, '--' , color='black',lw=0.5)
ax[0].set_xlim(-1.5,2)
ax[1].set_xlim(-1.5,2)
ax[2].set_xlim(-1.5,2)
ax[3].set_xlim(-1.5,2)
ax[0].set_ylim(-2,2.5)
ax[1].set_ylim(-2,2.5)
ax[2].set_ylim(-2,2.5)
ax[3].set_ylim(-2,2.5)
ax[0].set_xlabel(r'Log([OII]/Hβ)')
ax[1].set_xlabel(r'Log([OII]/Hβ)')
ax[2].set_xlabel(r'Log([OII]/Hβ)')
ax[3].set_xlabel(r'Log([OII]/Hβ)')
ax[0].set_ylabel(r'Log([OIII]5007/Hβ)')
plt.show()

# Fotometria

# (g-z) (u-r)

In [ ]:
ejexgz=df['g-z']
ejeyur=df['u-r']

fig, ax = plt.subplots(figsize=(8,6))

ax.scatter(ejexgz,ejeyur, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax.scatter(ejexgz,ejeyur, lin*1, marker='.',facecolors="none",edgecolors="orange",label='LINER')
ax.scatter(ejexgz,ejeyur, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax.scatter(ejexgz,ejeyur, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax.legend()
ax.set_xlim(-1,4)
ax.set_ylim(-1,5)
ax.set_xlabel(r'(g-z)') 
ax.set_ylabel(r'(u-r)')
plt.show()

fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].scatter(ejexgz,ejeyur, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax[1].scatter(ejexgz,ejeyur, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax[2].scatter(ejexgz,ejeyur, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax[3].scatter(ejexgz,ejeyur, lin*1, marker='.',facecolors="none",edgecolors="orange",label='Liner')

ax[0].set_xlim(-1,5)
ax[1].set_xlim(-1,5)
ax[2].set_xlim(-1,5)
ax[3].set_xlim(-1,5)
ax[0].set_ylim(-1,5)
ax[1].set_ylim(-1,5)
ax[2].set_ylim(-1,5)
ax[3].set_ylim(-1,5)
ax[0].set_xlabel(r'(g-z)') 
ax[1].set_xlabel(r'(g-z)') 
ax[2].set_xlabel(r'(g-z)') 
ax[3].set_xlabel(r'(g-z)') 
ax[0].set_ylabel(r'(u-r)')
plt.show()

# U-B

In [ ]:
x1 = np.linspace(-2,5)
a=1.4-(1.2*x1)
a[a <= -0.1] = -0.1

ejexub=df['U-B']
ejeyub=df['OIII/Hβ']

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(x1,a,'--', color='black',lw=1, label='Yan') 
ax.scatter(ejexub,ejeyub, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax.scatter(ejexub,ejeyub, lin*1, marker='.',facecolors="none",edgecolors="orange",label='LINER')
ax.scatter(ejexub,ejeyub, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax.scatter(ejexub,ejeyub, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax.scatter(train['U-B(T)'], train['OIII/Hβ(T)'], c="black", s=0.02,label='Training Data')
ax.legend()
ax.set_xlim(0,2)
ax.set_ylim(-1.5,1.5)
ax.set_xlabel(r'(U-B)',fontsize=16) 
ax.set_ylabel(r'Log([OIII]/Hβ)',fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.show()

fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].scatter(ejexub,ejeyub, sf*1, marker='.',facecolors="none",edgecolors="dodgerblue",label='Seyfert')
ax[1].scatter(ejexub,ejeyub, sb*1, marker='.',facecolors="none",edgecolors="red",label='Starburst')
ax[2].scatter(ejexub,ejeyub, comp*1, marker='.',facecolors="none",edgecolors="darkgreen",label='Comp') 
ax[3].scatter(ejexub,ejeyub, lin*1, marker='.',facecolors="none",edgecolors="orange",label='Liner')
ax[0].plot(x1,a,'--', color='black',lw=1) 
ax[1].plot(x1,a,'--', color='black',lw=1) 
ax[2].plot(x1,a,'--', color='black',lw=1) 
ax[3].plot(x1,a,'--', color='black',lw=1)
ax[0].set_xlim(0,2)
ax[1].set_xlim(0,2)
ax[2].set_xlim(0,2)
ax[3].set_xlim(0,2)
ax[0].set_ylim(-1.5,1.5)
ax[1].set_ylim(-1.5,1.5)
ax[2].set_ylim(-1.5,1.5)
ax[3].set_ylim(-1.5,1.5)
ax[0].set_xlabel(r'(U-B)') 
ax[1].set_xlabel(r'(U-B)') 
ax[2].set_xlabel(r'(U-B)') 
ax[3].set_xlabel(r'(U-B)') 
ax[0].set_ylabel(r'Log([OIII]/Hβ)')
plt.show()

# 3D Projection

In [ ]:
df['Predicted Class']=df['Predicted Class'].replace({0:'Comp', 1:'Liner', 2:'Seyfert', 3:'Starburst'})
comp=df['Predicted Class']=='Comp'
lin=df['Predicted Class']=='LINER'
sf=df['Predicted Class']=='Seyfert'
sb=df['Predicted Class']=='Starburst'

## EQW 

In [ ]:
%matplotlib qt
ejex1=df['W_Hα']
ejey1=df['NII/Hα']
ejez1=df['OIII/Hβ']
labels=df['Predicted Class']
df1 = pd.DataFrame(dict(x=ejex1, y=ejey1, z=ejez1, label=labels))

groups = df1.groupby('label')

color_palette = {'Comp': 'darkgreen', 'Liner': 'orange', 'Seyfert': 'dodgerblue', 'Starburst': 'red'}
plt.figure(figsize=(10,10))
ax=plt.axes(projection='3d')
for name, group in groups:
    color = color_palette.get(name, 'gray')
    ax.plot(group.x, group.y, group.z, marker='*', linestyle='', ms=1, label=name, color=color)
    
ax.scatter(train['W_Hα(T)'],train['NII/Hα(T)'], train['OIII/Hβ(T)'], c="black",s=0.02,label='Training Data') 
ax.set_xlim(-1,3)
ax.set_ylim(-2,1)
ax.set_zlim(-1.5,1.5)
ax.set_xlabel(r'Log($W_{Hα}$)') 
ax.set_ylabel(r'Log(NII/Hα)')
ax.set_zlabel(r'Log([OIII]5007/Hβ)')
ax.legend()
plt.show()

# U-B

In [ ]:
%matplotlib qt
ejex1=df['U-B']
ejey1=df['OII/Hβ']
ejez1=df['OIII/Hβ']
labels=df['Predicted Class']
df1 = pd.DataFrame(dict(x=ejex1, y=ejey1, z=ejez1, label=labels))

groups = df1.groupby('label')

color_palette = {'Comp': 'darkgreen', 'Liner': 'orange', 'Seyfert': 'dodgerblue', 'Starburst': 'red'}
plt.figure(figsize=(10,10))
ax=plt.axes(projection='3d')
for name, group in groups:
    color = color_palette.get(name, 'gray')
    ax.plot(group.x, group.y, group.z, marker='*', linestyle='', ms=1, label=name, color=color)
    
ax.scatter(train['U-B(T)'],train['OII/Hβ(T)'], train['OIII/Hβ(T)'], c="black",s=0.02,label='Training Data') 
ax.set_xlim(-0.5,2.5)
ax.set_ylim(-1,2)
ax.set_zlim(-1.5,2)
ax.set_xlabel(r'(U-B)') 
ax.set_ylabel(r'Log(OII/Hβ)')
ax.set_zlabel(r'Log([OIII]5007/Hβ)')
ax.legend()
plt.show()

## Color Color

In [ ]:
%matplotlib qt
ejex=df['g-z']
ejey=df['OIII/Hβ']
ejez=df['u-r']
labels=df['Predicted Class']
df4 = pd.DataFrame(dict(x=ejex, y=ejey, z=ejez, label=labels))

groups = df4.groupby('label')

color_palette = {'Comp': 'darkgreen', 'Liner': 'orange', 'Seyfert': 'dodgerblue', 'Starburst': 'red'}
plt.figure(figsize=(10,10))
ax=plt.axes(projection='3d')
for name, group in groups:
    color = color_palette.get(name, 'gray')
    ax.plot(group.x, group.y, group.z, marker='*', linestyle='', ms=1, label=name, color=color)

ax.scatter(train['g-z(T)'], train['OIII/Hβ(T)'], train['u-r(T)'], c="black",s=0.02,label='Training Data')
ax.set_xlim(0,2.5)
ax.set_ylim(-2,2)
ax.set_zlim(0,4)
ax.set_xlabel(r'(g-z)') 
ax.set_zlabel(r'(u-r))')
ax.set_ylabel(r'Log([OIII]5007/Hβ)')
ax.legend()
plt.show()

### Statistics 

In [ ]:
fig=px.box(df, x="U-B", y="Predicted Class", points="all")
fig.update_layout(xaxis_title="(U-B)", yaxis_title="Log([NII]/Hα)")

#df['mcolour'] = np.where(df['Predicted Class'] == "Starburst" , '#636EFA', 
     #np.where(df['Predicted Class'] == 'Seyfert', '#EF553B','#00CC96'))
#sns.catplot(data=df, x="U/Z", y="Predicted Class", kind="box",sym='',points="all")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sb = df[df['Predicted Class'] == "Starburst"]
sf = df[df['Predicted Class'] == "Seyfert"]
cm = df[df['Predicted Class'] == "Comp"]
li = df[df['Predicted Class'] == "Liner"]

ax=sns.kdeplot(sb['U-B'], sb['OIII/Hβ'], color="r",linestyles="--")
ax=sns.kdeplot(sf['U-B'], sf['OIII/Hβ'], color="g",linestyles="--")
ax=sns.kdeplot(cm['U-B'], cm['OIII/Hβ'], color="b",linestyles="--")
ax=sns.kdeplot(li['U-B'], li['OIII/Hβ'], color="orange",linestyles="--")
plt.xlim(-1,2)
#plt.ylim(-3,2)
plt.show()
#sns.kdeplot(data=df, x="U/Z", y="NII/Hα", hue="Predicted Class")'''

In [ ]:
#g=sns.relplot(data=df, x="U-B", y="NII/Hα", hue="Predicted Class", col="Predicted Class", col_wrap=2,palette=["orange","b","r","g"])
#g.set_axis_labels("Log(U/Z)","Log([NII]/Hα)")

In [ ]:
%matplotlib inline
w=sns.factorplot('Predicted Class',data=df,kind="count")
custom_palette = ["red","green","orange","blue"]
w=sns.set_palette(custom_palette)

In [ ]:
sns.heatmap(df.corr(), annot=True) and plt.show()